Instance Variable:
* capital_base
* open_positions
* context object - to store all necessary variables for that algorithm

In [1]:
import info
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime as dt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import warnings
import os.path
#import lightgbm as lgb
from utils import *
from backtest_algos import *

warnings.filterwarnings("ignore")

path_pc = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/'
today = datetime.today()
print("Today's date:", today.strftime("%Y-%m-%d"))

Today's date: 2020-05-01


In [2]:
outcomes_new = load_csv(path_pc, 'outcomes_new_features_2020-04-17.csv')

In [3]:
list(outcomes_new.columns)

['(close/ema)-1',
 '(close/sma)-1',
 '1-lower/close',
 '50d_high',
 '50d_high_volume',
 'atr10',
 'atr10/atr100',
 'atr10/atr100_scaled50',
 'atr10/atr20',
 'atr10/atr20_scaled50',
 'atr100',
 'atr100_scaled50',
 'atr10_scaled50',
 'atr14',
 'atr14_scaled50',
 'atr20',
 'atr20_scaled50',
 'bbands50_lower',
 'bbands50_lower_slope',
 'bbands50_upper',
 'bbands50_upper_slope',
 'bull',
 'bull_ratio',
 'close',
 'close>50d_high',
 'close_pct_change_5_day',
 'close_scaled50',
 'close_slope',
 'close_vs_50d_high',
 'delta_atr10/atr100_10',
 'delta_atr10/atr100_10_scaled50',
 'delta_atr10/atr100_3',
 'delta_atr10/atr100_3_scaled50',
 'ema40',
 'ema50',
 'ema50_scaled50',
 'ema50_slope',
 'ema80',
 'high',
 'high_scaled50',
 'high_slope',
 'intraday_chg',
 'label',
 'label_v2',
 'log volume',
 'log volume_pct_chg_1',
 'log volume_pct_chg_1_scaled50',
 'log volume_pct_chg_2',
 'log volume_pct_chg_2_scaled50',
 'log volume_pct_chg_3',
 'log volume_pct_chg_3_scaled50',
 'log volume_pct_chg_4',
 '

In [4]:
for lookback in [5,10,20,30,40]:
    zscore = lambda x: (x - x.rolling(window=lookback, min_periods=int(lookback*.8)).mean())/x.rolling(window=lookback, min_periods=int(lookback*.8)).std()
    col = 'close'
    outcomes_new[col+'_scaled'+str(lookback)] = outcomes_new.groupby(level='symbol')[col].apply(zscore)

### Result DataFrame
* Index = date
* columns = porfolio value, orders, positions (as dict)

### Change algo to include multiple symbols:
* Use last date of outcomes_new_dropna as latest test date
* backcount from there and get a list of test dates
* For each test date:
    * At Open, buy stocks from buy list.
        * Calculate best money allocation, with current cash amount
    * At close, sell stocks that hit day limit
    * Loop thru list of symbols:
        * Slice outcomes_new_dropna so that it only contain that symbol
        * Get list of trade dates from one-symbol dataframe
             * Check if current test date is in this symbol's trade date list. If not, it means this symbol is not trade on                    that date. SKIPP THIS SYMBOL
        * From that list calculate training start, valid start, etc.
        * Train and make prediction. Add predictions to stock buy list

def backtest_rf(outcomes_new):
    
    outcomes_new_dropna = outcomes_new.dropna()
    
    trade_date_list = sorted(list(set(outcomes_new.index.get_level_values('date'))))
    
    features_selected = info.features_lgb_v2 # + ['close_scaled5','close_scaled10','close_scaled20','close_scaled30','close_scaled40']
    
    max_impact_pct = 0.001
    max_impact_volatility = 'price_chg_1_ema_std50'
    total_budget = context.capital_base * 1.2
    
    test_period = 250
    test_size = 1
    valid_size = 1
    training_size = 2000
    valid_test_gap = 4
    label = 'label_v2'

    #make a list of test dates according to test period, by back counting from the latest date with target
    test_dates = sorted(list(set(outcomes_new_dropna.index.get_level_values('date'))))[-test_period:]
    
    test_dates_converted = []
    for ts in test_dates:
        test_dates_converted.append(ts.to_pydatetime())
    
    results = pd.DataFrame()
    
    context.predictions = pd.DataFrame()
    
    for test_date in test_dates_converted:
        
        print()
        
        price_list = get_price_list(outcomes_new, test_date)
        
        price_list_open = get_price_list_open(outcomes_new, test_date)
        
        #update positions value according to open prices
        context.positions_value = positions_value(context.positions, price_list_open)
        
        if 'queue' not in globals() and 'queue' not in locals():
            queue = pd.DataFrame()
        
        context.max_positions = int((context.positions_value+context.cash)/(context.typical_amount * \
                                        (context.margin_ratio_long+context.margin_ratio_short)/2))
        context.max_daily_orders = int(context.max_positions/context.average_holding_period)
        context.max_daily_margin_limit = (context.positions_value+context.cash)/context.average_holding_period
        
        remaining_orders = context.max_daily_orders
        remaining_margin_limit = context.max_daily_margin_limit
        
        if len(queue)>0:
            while True:
                if len(queue)==0 or remaining_orders==0 or remaining_margin_limit<=0:
                    print ('Break with len(queue) = {}, remaining_orders = {} and remaining_margin_limit = {}'.format(len(queue),
                                                                                                                          remaining_orders,
                                                                                                                          remaining_margin_limit))
                    break
                order_list = list(queue[:min(remaining_orders, len(queue))]['symbol'])
                yesterday = trade_date_list[trade_date_list.index(test_date)-1]
                weights = inverse_volatility_weights(order_list, yesterday, outcomes_new)
                
                #place order
                symbol = queue[:1]['symbol'].values[0]
                order_type = queue[:1]['action'].values[0]
                order_id = ''
                num_shares = 1
                date_open = test_date
                traded_price = price_list_open[:,symbol].values[0]
                date_limit = trade_date_list[trade_date_list.index(date_open)+4]
                target_up = outcomes_new.loc[(date_open,symbol)]['target_upper_v2']
                target_down = outcomes_new.loc[(date_open,symbol)]['target_lower_v2']
                percentage = weights[symbol] if len(order_list)>1 else 1
                print()
                print("[{}] Symbol: {} Date open: {}, Date limit: {}, Traded price: {}.".format(order_type,
                                                                                                symbol,
                                                                                                date_open.strftime("%Y-%m-%d"),
                                                                                                date_limit.strftime("%Y-%m-%d"),
                                                                                                traded_price))
                order = Order(order_type, symbol, num_shares, date_open, traded_price, date_limit, target_up, target_down)
                status, margin_change = order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                if status == 'successful':
                    remaining_orders-=1
                    queue=queue.iloc[1:]
                    remaining_margin_limit-=margin_change
                    
                elif status == 'margin_overshoot':
                    break
                        
                elif status == 'min_charge_not_hitting':
                    #Cancel order
                    queue=queue.iloc[1:]

        #listen to market price of all stocks in positions
        #If hitting target price, then sell
        for i, row in context.positions.iterrows():
            if outcomes_new.loc[(test_date,row['symbol'])]['high']>row['target_up']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = row['target_up']
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                continue

            if outcomes_new.loc[(test_date,row['symbol'])]['low']<row['target_down']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = row['target_down']
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                continue
        
        #Near market close - check what stock in position, with date limit = today. Sell it at close price
        for i, row in context.positions.iterrows():
            #print('type of row[date_limit]', type(row['date_limit']))
            if test_date >= row['date_limit']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = price_list[:,row['symbol']].values[0]
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
        
        
        #After all the buy and sell (imagine the time now is after market close)
        #Update porfolio value
        context.positions_value = positions_value(context.positions, price_list)
        
        if 'symbol' in context.positions.columns:
            positions_list = list(context.positions['symbol'])
        else:
            positions_list = []
                                  
        result = {'positions': positions_list, #context.positions.to_dict('records'),
                'date': test_date,
                 'positions_value': context.positions_value,
                 'cash_value': context.cash,
                 'porfolio_value': context.positions_value+context.cash}
        
        results = results.append(result, ignore_index=True)
        
        #Update margin value due to the closing positions and STOCK VALUE CHANGE. USE CLOSE VALUE FOR THE DAY
        context.current_margin = positions_value_absolute(context.positions, price_list) * context.margin_ratio_long
        
        #CHECK MAINTENANCE MARGIN
        if context.positions_value+context.cash <= context.current_margin*0.8:
            print ('ELV lower than!!! MAINTENANCE MARGIN. GAME OVER LIAO.')
            return results

        #End of Day SMA calculation: ELV-Reg T Margin >= 0, cash>=-0.5*stock_value
        if context.positions_value+context.cash < context.positions_value * 0.5:
            print ('SMA smaller than zero! GAME OVER.')
            return results
        
        #HERE's the PREDICTION PART

        print('Test date: ', test_date)

        #get all trading dates from dataset of this symbol
        data_dates = sorted(list(set(outcomes_new_dropna.index.get_level_values(0))))
        data_converted_dates = []
        for ts in data_dates:
            data_converted_dates.append(ts.to_pydatetime())

        #Calculate training start date and valid start date by back counting
        start_date = data_converted_dates[data_converted_dates.index(test_date)-(valid_size + training_size + valid_test_gap + test_size)]
        start_date_valid = data_converted_dates[data_converted_dates.index(test_date)-(valid_size + valid_test_gap + test_size)]
        end_date_valid = data_converted_dates[data_converted_dates.index(test_date)-valid_test_gap-test_size]
        start_date_test = data_converted_dates[data_converted_dates.index(test_date)-test_size]
        end_date_test = test_date

        X_y_train, X_y_valid, X_y_test = train_valid_test_split(outcomes_new_dropna, start_date,
                                                                    start_date_valid, end_date_valid, start_date_test, end_date_test)

        #downsample the training set so each class has same number of samples
        X_y_train_resampled = downsample_3class(X_y_train, label, 42)

        #rename all -1 labels to 2 as LGB only takes 0,1,2...
        X_y_train_resampled.loc[X_y_train_resampled[label] == -1, label] = 2
        X_y_valid.loc[X_y_valid[label] == -1, label] = 2
        X_y_test.loc[X_y_test[label] == -1, label] = 2

        #split into features and target sets
        X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, label)
        X_valid, y_valid = feature_target_split(X_y_valid, features_selected, label)
        X_test, y_test = feature_target_split(X_y_test, features_selected, label)

        model_rf =RandomForestClassifier(random_state=42, min_samples_split=8,
                                      min_samples_leaf=2, max_features=0.4780494942262089,
                                      n_estimators=172, max_leaf_nodes=1000, bootstrap=True, n_jobs=4)
        model_rf.fit(X_train, y_train)

        y_class_pred = model_rf.predict(X_test)

        #y_class_pred = [pred.argmax() for pred in y_pred]

        X_test['pred'] = y_class_pred
    
        buy_df = pd.DataFrame()
        short_df = pd.DataFrame()
        
        ##################################################
        ##################################################
        #search criteria for coming up with buy/short list
        search_criteria = 'momentum_50'
        momentum_filter = 'close_scaled5'
        ##################################################
        ##################################################
        
        for i, row in X_test.iterrows():
            if row['pred'] == 1:
                entry = {'symbol': i[1],
                        'volatility': row['past_return_5_std50'],
                        search_criteria: row[search_criteria]}
                buy_df=buy_df.append(entry, ignore_index=True)
                
            elif row['pred'] == 2: # and row[momentum_filter]<0:
                entry = {'symbol': i[1],
                        'volatility': row['past_return_5_std50'],
                       search_criteria: row[search_criteria]}
                short_df=short_df.append(entry, ignore_index=True)
                
        if len(buy_df)!=0 or len(short_df)!=0:
            positive_pct = len(buy_df)/(len(buy_df)+len(short_df))
            negative_pct = len(short_df)/(len(buy_df)+len(short_df))
        else:
            positive_pct = 0
            negative_pct = 0
        
        print('Postive percentage: ', positive_pct)
        print('Negative percentage: ', negative_pct)
        print()
        
        if len(buy_df)>0:
            positive_list = list(buy_df['symbol'])
        else:
            positive_list = []
            
        if len(short_df)>0:
            negative_list = list(short_df['symbol'])
        else:
            negative_list = []
            
        predictions_entry = {'date':test_date,
                            'num_predictions_long':len(buy_df),
                            'num_predictions_short':len(short_df),
                            'predictions_long':positive_list,
                            'predictions_short':negative_list,
                            'percentage_long':positive_pct,
                            'percentage_short':negative_pct}
        
        context.predictions=context.predictions.append(predictions_entry, ignore_index=True)
        
        buy_drop_indexes = []
        short_drop_indexes = []
        
        for i, row in buy_df.iterrows():
            if len(context.positions)>0:
                if row['symbol'] in list(context.positions['symbol']):
                    buy_drop_indexes.append(i)

        for i, row in short_df.iterrows():
            if len(context.positions)>0:
                if row['symbol'] in list(context.positions['symbol']):
                    short_drop_indexes.append(i)
        
        buy_df=buy_df.drop(buy_drop_indexes)
        short_df=short_df.drop(short_drop_indexes)
        
        search_criteria = 'volatility'
        
        buy_list = []
        short_list = []

        if len(buy_df)>0:
            buy_df = buy_df.sort_values(by=[search_criteria], ascending=True)
            buy_list = list(buy_df['symbol'])
        
        if len(short_df)>0:
            short_df = short_df.sort_values(by=[search_criteria], ascending=True)
            short_list = list(short_df['symbol'])
            
        print("--------------------------------")
        print('Buy_list symbols:')
        print(buy_list)
        print('Short_list symbols:')
        print(short_list)
        print("--------------------------------")
        
        len_buy = len(buy_list)
        len_short = len(short_list)
        
        queue = pd.DataFrame()
        
        #arrange buy and short list in a queue, alternately buy and short
        for symbol_to_buy, symbol_to_short in zip(buy_list, short_list):
            queue = queue.append({'symbol':symbol_to_buy,
                                  'action':'BUY'}, ignore_index=True)
            queue = queue.append({'symbol':symbol_to_short,
                                  'action':'SHORT'}, ignore_index=True)
        if len_buy>len_short:
            for k in range(len_short, len_buy):
                queue = queue.append({'symbol':buy_list[k],
                                      'action':'BUY'}, ignore_index=True)
        elif len_short>len_buy:
            for k in range(len_buy, len_short):
                queue = queue.append({'symbol':short_list[k],
                                      'action':'SHORT'}, ignore_index=True)
        print()
        print(queue)

    return results

In [5]:
def backtest_lgb_v3(outcomes_new):
    
    retrain_interval = 50
    
    outcomes_new_dropna = outcomes_new.dropna()
    
    trade_date_list = sorted(list(set(outcomes_new.index.get_level_values('date'))))
    
    momentum_filter = 'close_scaled10'
    
    features_selected = info.features_lgb_v2  + [momentum_filter] #,'close_scaled10','close_scaled20','close_scaled30','close_scaled40']
    
    max_impact_pct = 0.001
    max_impact_volatility = 'price_chg_1_ema_std50'
    total_budget = context.capital_base * 1.2
    
    test_period = 250
    test_size = 1
    valid_size = 1
    training_size = 2000
    valid_test_gap = 4
    label = 'label_v2'

    #make a list of test dates according to test period, by back counting from the latest date with target
    test_dates = sorted(list(set(outcomes_new_dropna.index.get_level_values('date'))))[-test_period:]
    
    test_dates_converted = []
    for ts in test_dates:
        test_dates_converted.append(ts.to_pydatetime())
    
    results = pd.DataFrame()
    
    context.predictions = pd.DataFrame()
    
    day_count = 0
    
    for test_date in test_dates_converted:
        

        
        print()
        
        price_list = get_price_list(outcomes_new, test_date)
        
        price_list_open = get_price_list_open(outcomes_new, test_date)
        
        #update positions value according to open prices
        context.positions_value = positions_value(context.positions, price_list_open)
        
        if 'queue' not in globals() and 'queue' not in locals():
            queue = pd.DataFrame()
        
        context.max_positions = int((context.positions_value+context.cash)/(context.typical_amount * \
                                        (context.margin_ratio_long+context.margin_ratio_short)/2))
        context.max_daily_orders = int(context.max_positions/context.average_holding_period)
        context.max_daily_margin_limit = (context.positions_value+context.cash)/context.average_holding_period
        
        remaining_orders = context.max_daily_orders
        remaining_margin_limit = context.max_daily_margin_limit
        
        if len(queue)>0:
            while True:
                if len(queue)==0 or remaining_orders==0 or remaining_margin_limit<=0:
                    print ('Break with len(queue) = {}, remaining_orders = {} and remaining_margin_limit = {}'.format(len(queue),
                                                                                                                          remaining_orders,
                                                                                                                          remaining_margin_limit))
                    break
                order_list = list(queue[:min(remaining_orders, len(queue))]['symbol'])
                yesterday = trade_date_list[trade_date_list.index(test_date)-1]
                weights = inverse_volatility_weights(order_list, yesterday, outcomes_new)
                
                #place order
                symbol = queue[:1]['symbol'].values[0]
                order_type = queue[:1]['action'].values[0]
                order_id = ''
                num_shares = 1
                date_open = test_date
                traded_price = price_list_open[:,symbol].values[0]
                date_limit = trade_date_list[trade_date_list.index(date_open)+4]
                target_up = outcomes_new.loc[(date_open,symbol)]['target_upper_v2']
                target_down = outcomes_new.loc[(date_open,symbol)]['target_lower_v2']
                percentage = weights[symbol] if len(order_list)>1 else 1
                print()
                print("[{}] Symbol: {} Date open: {}, Date limit: {}, Traded price: {}.".format(order_type,
                                                                                                symbol,
                                                                                                date_open.strftime("%Y-%m-%d"),
                                                                                                date_limit.strftime("%Y-%m-%d"),
                                                                                                traded_price))
                order = Order(order_type, symbol, num_shares, date_open, traded_price, date_limit, target_up, target_down)
                status, margin_change = order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                if status == 'successful':
                    remaining_orders-=1
                    queue=queue.iloc[1:]
                    remaining_margin_limit-=margin_change
                    
                elif status == 'margin_overshoot':
                    break
                        
                elif status == 'min_charge_not_hitting':
                    #Cancel order
                    queue=queue.iloc[1:]

        #listen to market price of all stocks in positions
        #If hitting target price, then sell
        for i, row in context.positions.iterrows():
            if outcomes_new.loc[(test_date,row['symbol'])]['high']>row['target_up']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = row['target_up']
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                continue

            if outcomes_new.loc[(test_date,row['symbol'])]['low']<row['target_down']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = row['target_down']
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
                continue
        
        #Near market close - check what stock in position, with date limit = today. Sell it at close price
        for i, row in context.positions.iterrows():
            #print('type of row[date_limit]', type(row['date_limit']))
            if test_date >= row['date_limit']:
                order_type = 'CLOSE'
                order_id = row['order_id']
                num_lots = 1
                num_shares = float('Nan')
                date_open = float('Nan')
                traded_price = price_list[:,row['symbol']].values[0]
                date_limit = float('Nan')
                target_up = float('Nan')
                target_down = float('Nan')
                percentage = 0
                
                print()
                print("Making CLOSE order.", test_date.strftime("%Y-%m-%d"), traded_price)
                order = Order(order_type, row['symbol'], num_shares, date_open, traded_price, date_limit, target_up, target_down)
                order_target_percent(order,  percentage, order_id, context, remaining_margin_limit)
        
        
        #After all the buy and sell (imagine the time now is after market close)
        #Update porfolio value
        context.positions_value = positions_value(context.positions, price_list)
        
        if 'symbol' in context.positions.columns:
            positions_list = list(context.positions['symbol'])
        else:
            positions_list = []
                                  
        result = {'positions': positions_list, #context.positions.to_dict('records'),
                'date': test_date,
                 'positions_value': context.positions_value,
                 'cash_value': context.cash,
                 'porfolio_value': context.positions_value+context.cash}
        
        results = results.append(result, ignore_index=True)
        
        #Update margin value due to the closing positions and STOCK VALUE CHANGE. USE CLOSE VALUE FOR THE DAY
        context.current_margin = positions_value_absolute(context.positions, price_list) * context.margin_ratio_long
        
        #CHECK MAINTENANCE MARGIN
        if context.positions_value+context.cash <= context.current_margin*0.8:
            print ('ELV lower than!!! MAINTENANCE MARGIN. GAME OVER LIAO.')
            return results

        #End of Day SMA calculation: ELV-Reg T Margin >= 0, cash>=-0.5*stock_value
        if context.positions_value+context.cash < context.positions_value * 0.5:
            print ('SMA smaller than zero! GAME OVER.')
            return results
        
        #HERE's the PREDICTION PART
        print('Test date: ', test_date)

        #get all trading dates from dataset of this symbol
        data_dates = sorted(list(set(outcomes_new_dropna.index.get_level_values(0))))
        data_converted_dates = []
        for ts in data_dates:
            data_converted_dates.append(ts.to_pydatetime())

        #Calculate training start date and valid start date by back counting
        start_date = data_converted_dates[data_converted_dates.index(test_date)-(valid_size + training_size + valid_test_gap + test_size)]
        start_date_valid = data_converted_dates[data_converted_dates.index(test_date)-(valid_size + valid_test_gap + test_size)]
        end_date_valid = data_converted_dates[data_converted_dates.index(test_date)-valid_test_gap-test_size]
        start_date_test = data_converted_dates[data_converted_dates.index(test_date)-test_size]
        end_date_test = test_date

        X_y_train, X_y_valid, X_y_test = train_valid_test_split(outcomes_new_dropna, start_date,
                                                                    start_date_valid, end_date_valid, start_date_test, end_date_test)

        #downsample the training set so each class has same number of samples
        X_y_train_resampled = downsample_3class(X_y_train, label, 42)

        #rename all -1 labels to 2 as LGB only takes 0,1,2...
        X_y_train_resampled.loc[X_y_train_resampled[label] == -1, label] = 2
        X_y_valid.loc[X_y_valid[label] == -1, label] = 2
        X_y_test.loc[X_y_test[label] == -1, label] = 2

        #split into features and target sets
        X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, label)
        X_valid, y_valid = feature_target_split(X_y_valid, features_selected, label)
        X_test, y_test = feature_target_split(X_y_test, features_selected, label)

        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
        
        max_acc = float("-inf")
        boosting = 'gbdt'

        learning_rate = 0.006985117638031729
        min_data = 16
        num_leaves = 47
        max_depth = 7

        parameters = {
            'application': 'multiclass',
            'num_class': 3,
            #'is_unbalance': 'false',
            #'metric': 'auc',
            #'scale_pos_weight': 9,
            'boosting': boosting,
            'num_leaves': num_leaves,
            'feature_fraction': 0.95,
            'bagging_fraction': 0.2,
            'bagging_freq': 10,
            'learning_rate': learning_rate,
            'verbose': 0,
            'min_data_in_leaf': min_data,
            'max_depth': max_depth
        }
        
#         if day_count%50 == 0:
        model = lgb.train(parameters,
                               train_data,
                               valid_sets=valid_data,
                               num_boost_round=5000,
                                verbose_eval=False,
                               #feval=lgb_f1_score,
                               early_stopping_rounds=100)
            
        
        day_count += 1
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        y_class_pred = [pred.argmax() for pred in y_pred]

        X_test['pred'] = y_class_pred
        buy_df = pd.DataFrame()
        short_df = pd.DataFrame()
        
        ##################################################
        ##################################################
        #search criteria for coming up with buy/short list
        search_criteria = 'momentum_25'
        ##################################################
        ##################################################
        
        for i, row in X_test.iterrows():
#             if row['pred'] == 1 and row[momentum_filter]>0:
#                 entry = {'symbol': i[1],
#                         'volatility': row['past_return_5_std50'],
#                         search_criteria: row[search_criteria]}
#                 buy_df=buy_df.append(entry, ignore_index=True)
                
            if row['pred'] == 2 and row[momentum_filter]<-0.5:
                entry = {'symbol': i[1],
                        'volatility': row['past_return_5_std50'],
                       search_criteria: row[search_criteria]}
                short_df=short_df.append(entry, ignore_index=True)
                
        if len(buy_df)!=0 or len(short_df)!=0:
            positive_pct = len(buy_df)/(len(buy_df)+len(short_df))
            negative_pct = len(short_df)/(len(buy_df)+len(short_df))
        else:
            positive_pct = 0
            negative_pct = 0
        
        print('Postive percentage: ', positive_pct)
        print('Negative percentage: ', negative_pct)
        print()
        
        if len(buy_df)>0:
            positive_list = list(buy_df['symbol'])
        else:
            positive_list = []
            
        if len(short_df)>0:
            negative_list = list(short_df['symbol'])
        else:
            negative_list = []
            
        predictions_entry = {'date':test_date,
                            'num_predictions_long':len(buy_df),
                            'num_predictions_short':len(short_df),
                            'predictions_long':positive_list,
                            'predictions_short':negative_list,
                            'percentage_long':positive_pct,
                            'percentage_short':negative_pct}
        
        context.predictions=context.predictions.append(predictions_entry, ignore_index=True)
        
        buy_drop_indexes = []
        short_drop_indexes = []
        
        for i, row in buy_df.iterrows():
            if len(context.positions)>0:
                if row['symbol'] in list(context.positions['symbol']):
                    buy_drop_indexes.append(i)

        for i, row in short_df.iterrows():
            if len(context.positions)>0:
                if row['symbol'] in list(context.positions['symbol']):
                    short_drop_indexes.append(i)
        
        buy_df=buy_df.drop(buy_drop_indexes)
        short_df=short_df.drop(short_drop_indexes)
        
        #search_criteria = 'volatility'
        
        buy_list = []
        short_list = []

        if len(buy_df)>0:
            buy_df = buy_df.sort_values(by=[search_criteria], ascending=False)
            buy_list = list(buy_df['symbol'])
        
        if len(short_df)>0:
            short_df = short_df.sort_values(by=[search_criteria], ascending=True)
            short_list = list(short_df['symbol'])
            
        print("--------------------------------")
        print('Buy_list symbols:')
        print(buy_list)
        print('Short_list symbols:')
        print(short_list)
        print("--------------------------------")
        
        len_buy = len(buy_list)
        len_short = len(short_list)
        
        queue = pd.DataFrame()
        
        #arrange buy and short list in a queue, alternately buy and short
        for symbol_to_buy, symbol_to_short in zip(buy_list, short_list):
            queue = queue.append({'symbol':symbol_to_buy,
                                  'action':'BUY'}, ignore_index=True)
            queue = queue.append({'symbol':symbol_to_short,
                                  'action':'SHORT'}, ignore_index=True)
        if len_buy>len_short:
            for k in range(len_short, len_buy):
                queue = queue.append({'symbol':buy_list[k],
                                      'action':'BUY'}, ignore_index=True)
        elif len_short>len_buy:
            for k in range(len_buy, len_short):
                queue = queue.append({'symbol':short_list[k],
                                      'action':'SHORT'}, ignore_index=True)
        print()
        print(queue)

    return results

### Consolidate the buy/short list to one long buy/Short alternated list with ALL PREDICTIONS
#### Policy to deal with not enough money to get 1 lot. If targeted amount is more than 0.85 of min_charge of that stock, make it one lot. Else drop this order

Queue should have: 'BUY/SHORT', 'symbol', 'volatility', 

Next day start:


update max_positions
update max_daily_orders
update max_daily_budget
update min_charges dictionary
remaining_daily_buget = max_daily_budget
remaining_orders = max_daily_orders

Set while loop to place order in the order of queue.
* Break if counter meets remaining_orders==0 or len(queue)==0
* Do inverse volatility on the first min(remaining_orders, len(queue)) items, USING YESTERDAYS' CLOSE, remaining_daily_buget
* Place order.
    * If successful, remaining_orders-=1, queue.pop(current_item), remaining_daily_budget-=amount, continue loop
    * If unsuccessful
        * If it is margin overshoot, break loop
        * If it is not hitting min_charge
            * If amount/min_charge > lot_threshold_fraction:
                * Re-adjust the percentage to min_charge/remaining_daily_buget, place order again, remaining_orders-=1, queue.pop(current_item), remaining_daily_budget-=amount, continue loop 
            * Else: 
                * queue.pop(current_item), continue loop

### Volatility Parity Sizing in Core Trend Following model
#### Number of shares to buy = Risk_Factor * Total_Porfolio_Value/Std
So that every position gives you same daily impact to the porfolio.

### 3 day settlement rule?

In [6]:
context = Context()
initialize(context)

In [7]:
import time
#import random

start_time = time.time()

results = backtest_lgb_v3(outcomes_new)
#results = backtest_random(outcomes_new, context)
#results = backtest_rf(outcomes_new)

print("--- %s seconds ---" % (time.time() - start_time))
print(str(dt.timedelta(seconds=(time.time() - start_time))))


Test date:  2019-03-29 00:00:00
Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['0386.HK', '1088.HK', '0939.HK', '0941.HK']
--------------------------------

  action   symbol
0  SHORT  0386.HK
1  SHORT  1088.HK
2  SHORT  0939.HK
3  SHORT  0941.HK


[SHORT] Symbol: 0386.HK Date open: 2019-04-01, Date limit: 2019-04-08, Traded price: 6.31.
Amount needed =  63099.99999999999
Current cash =  500000
Margin change =  31549.999999999996
Cash balance =  563100.0
Current margin:  31549.999999999996

[SHORT] Symbol: 1088.HK Date open: 2019-04-01, Date limit: 2019-04-08, Traded price: 17.98.
Amount needed =  62930.0
Current cash =  563100.0
Margin change =  31465.0
Cash balance =  626030.0
Current margin:  63015.0

[SHORT] Symbol: 0939.HK Date open: 2019-04-01, Date limit: 2019-04-08, Traded price: 6.8.
Amount needed =  122400.0
Current cash =  626030.0
Margin change =  61200.0
Cash balance =  748430.0
Current margin

Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['0083.HK', '0267.HK', '0151.HK', '0019.HK', '0016.HK', '0017.HK']
--------------------------------

  action   symbol
0  SHORT  0083.HK
1  SHORT  0267.HK
2  SHORT  0151.HK
3  SHORT  0019.HK
4  SHORT  0016.HK
5  SHORT  0017.HK


[SHORT] Symbol: 0083.HK Date open: 2019-04-16, Date limit: 2019-04-24, Traded price: 14.14.
Amount needed =  56560.0
Current cash =  1225216.8047946924
Margin change =  28280.0
Cash balance =  1281776.8047946924
Current margin:  381790.4

[SHORT] Symbol: 0267.HK Date open: 2019-04-16, Date limit: 2019-04-24, Traded price: 11.76.
Amount needed =  82320.0
Current cash =  1281776.8047946924
Margin change =  41160.0
Cash balance =  1364096.8047946924
Current margin:  422950.4

[SHORT] Symbol: 0151.HK Date open: 2019-04-16, Date limit: 2019-04-24, Traded price: 6.4.
Amount needed =  57600.0
Current cash =  1364096.8047946924
Margin change =  

Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['0151.HK', '0066.HK', '0003.HK']
--------------------------------

  action   symbol
0  SHORT  0151.HK
1  SHORT  0066.HK
2  SHORT  0003.HK


[SHORT] Symbol: 0151.HK Date open: 2019-04-26, Date limit: 2019-05-03, Traded price: 6.21.
Amount needed =  93150.0
Current cash =  1210894.1762758892
Margin change =  46575.0
Cash balance =  1304044.1762758892
Current margin:  384375.0

[SHORT] Symbol: 0066.HK Date open: 2019-04-26, Date limit: 2019-05-03, Traded price: 46.6.
Amount needed =  116500.0
Current cash =  1304044.1762758892
Margin change =  58250.0
Cash balance =  1420544.1762758892
Current margin:  442625.0

[SHORT] Symbol: 0003.HK Date open: 2019-04-26, Date limit: 2019-05-03, Traded price: 16.8909.
Amount needed =  135127.19999999998
Current cash =  1420544.1762758892
Margin change =  67563.59999999999
Cash balance =  1555671.376275889
Current margin:  5101


[SHORT] Symbol: 0941.HK Date open: 2019-05-07, Date limit: 2019-05-14, Traded price: 73.5.
Amount needed =  36750.0
Current cash =  1110045.1807396475
Margin change =  18375.0
Cash balance =  1146795.1807396475
Current margin:  301390.0

[SHORT] Symbol: 0386.HK Date open: 2019-05-07, Date limit: 2019-05-14, Traded price: 5.91.
Amount needed =  35460.0
Current cash =  1146795.1807396475
Margin change =  17730.0
Cash balance =  1182255.1807396475
Current margin:  319120.0

[SHORT] Symbol: 0857.HK Date open: 2019-05-07, Date limit: 2019-05-14, Traded price: 4.84.
Amount needed =  29040.0
Current cash =  1182255.1807396475
Margin change =  14520.0
Cash balance =  1211295.1807396475
Current margin:  333640.0

[SHORT] Symbol: 0066.HK Date open: 2019-05-07, Date limit: 2019-05-14, Traded price: 47.0.
Amount needed =  47000.0
Current cash =  1211295.1807396475
Margin change =  23500.0
Cash balance =  1258295.1807396475
Current margin:  357140.0

[SHORT] Symbol: 0002.HK Date open: 2019-05-07, 


[SHORT] Symbol: 0267.HK Date open: 2019-05-10, Date limit: 2019-05-17, Traded price: 11.16.
Amount needed =  55800.0
Current cash =  1234491.0261943473
Margin change =  27900.0
Cash balance =  1290291.0261943473
Current margin:  362397.7

[SHORT] Symbol: 0857.HK Date open: 2019-05-10, Date limit: 2019-05-17, Traded price: 4.66.
Amount needed =  37280.0
Current cash =  1290291.0261943473
Margin change =  18640.0
Cash balance =  1327571.0261943473
Current margin:  381037.7

[SHORT] Symbol: 1109.HK Date open: 2019-05-10, Date limit: 2019-05-17, Traded price: 32.65.
Percentage of captital is not enough for 1 lot of 1109.HK.
ORDER CANCELLED.

[SHORT] Symbol: 0883.HK Date open: 2019-05-10, Date limit: 2019-05-17, Traded price: 13.6.
Amount needed =  27200.0
Current cash =  1327571.0261943473
Margin change =  13600.0
Cash balance =  1354771.0261943473
Current margin:  394637.7

[SHORT] Symbol: 0388.HK Date open: 2019-05-10, Date limit: 2019-05-17, Traded price: 251.0.
Amount needed =  25100.

...

Making CLOSE order. 2020-03-02 9.85
order_id =  1418.0
indext to drop =  4
Cash gain:  -39400.0
Balance:  1477527.4867350052

Making CLOSE order. 2020-03-02 109.0
order_id =  1420.0
indext to drop =  5
Cash gain:  -54500.0
Balance:  1423027.4867350052
Test date:  2020-03-02 00:00:00
Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['0883.HK', '1928.HK', '0066.HK', '0027.HK', '0002.HK', '2388.HK', '0700.HK', '0288.HK', '0175.HK']
--------------------------------

  action   symbol
0  SHORT  0883.HK
1  SHORT  1928.HK
2  SHORT  0066.HK
3  SHORT  0027.HK
4  SHORT  0002.HK
5  SHORT  2388.HK
6  SHORT  0700.HK
7  SHORT  0288.HK
8  SHORT  0175.HK


[SHORT] Symbol: 0883.HK Date open: 2020-03-03, Date limit: 2020-03-09, Traded price: 11.08.
Amount needed =  22160.0
Current cash =  1423027.4867350052
Margin change =  11080.0
Cash balance =  1445187.4867350052
Current margin:  412795.0

[SHORT] Symbol: 1928.HK Date open: 

Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['1997.HK']
--------------------------------

  action   symbol
0  SHORT  1997.HK


[SHORT] Symbol: 1997.HK Date open: 2020-03-06, Date limit: 2020-03-12, Traded price: 35.9.
Amount needed =  394900.0
Current cash =  1310692.0535044796
Margin change =  197450.0
Cash balance =  1705592.0535044796
Current margin:  551965.0
Break with len(queue) = 0, remaining_orders = 9 and remaining_margin_limit = 6279.017834826518

Making CLOSE order. 2020-03-06 3.952126916945636
order_id =  1448.0
indext to drop =  1
Cash gain:  -39521.26916945636
Balance:  1666070.7843350233

Making CLOSE order. 2020-03-06 47.818890275231546
order_id =  1487.0
indext to drop =  14
Cash gain:  -23909.445137615774
Balance:  1642161.3391974075

Making CLOSE order. 2020-03-06 3.01
order_id =  1445.0
indext to drop =  0
Cash gain:  -30099.999999999996
Balance:  1612061.3391974075

Making CLOSE orde

Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['0001.HK', '0017.HK', '0002.HK']
--------------------------------

  action   symbol
0  SHORT  0001.HK
1  SHORT  0017.HK
2  SHORT  0002.HK


[SHORT] Symbol: 0001.HK Date open: 2020-03-11, Date limit: 2020-03-17, Traded price: 63.3.
Amount needed =  94950.0
Current cash =  1422792.596339318
Margin change =  47475.0
Cash balance =  1517742.596339318
Current margin:  446912.5

[SHORT] Symbol: 0017.HK Date open: 2020-03-11, Date limit: 2020-03-17, Traded price: 9.59.
Amount needed =  76720.0
Current cash =  1517742.596339318
Margin change =  38360.0
Cash balance =  1594462.596339318
Current margin:  485272.5

[SHORT] Symbol: 0002.HK Date open: 2020-03-11, Date limit: 2020-03-17, Traded price: 80.85.
Amount needed =  242549.99999999997
Current cash =  1594462.596339318
Margin change =  121274.99999999999
Cash balance =  1837012.596339318
Current margin:  606547.5
Br

Balance:  960032.7840017925

Making CLOSE order. 2020-03-13 8.422682613426238
order_id =  1562.0
indext to drop =  7
Cash gain:  -42113.41306713119
Balance:  917919.3709346614

Making CLOSE order. 2020-03-13 349.1761480594865
order_id =  1563.0
indext to drop =  8
Cash gain:  -34917.61480594865
Balance:  883001.7561287127

Making CLOSE order. 2020-03-13 55.64186655948455
order_id =  1564.0
indext to drop =  9
Cash gain:  -27820.933279742272
Balance:  855180.8228489704

Making CLOSE order. 2020-03-13 5.3574090649998425
order_id =  1566.0
indext to drop =  11
Cash gain:  -75003.7269099978
Balance:  780177.0959389727
Test date:  2020-03-13 00:00:00
Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['1093.HK', '0006.HK', '0003.HK', '0002.HK']
--------------------------------

  action   symbol
0  SHORT  1093.HK
1  SHORT  0006.HK
2  SHORT  0003.HK
3  SHORT  0002.HK


[SHORT] Symbol: 1093.HK Date open: 2020-03-16, Da

order_id =  1598.0
indext to drop =  9
Cash gain:  -22636.65439480597
Balance:  985933.9785423956

Making CLOSE order. 2020-03-19 81.82574048574794
order_id =  1600.0
indext to drop =  11
Cash gain:  -49095.444291448766
Balance:  936838.5342509468

Making CLOSE order. 2020-03-19 10.29282928272501
order_id =  1602.0
indext to drop =  13
Cash gain:  -61756.97569635006
Balance:  875081.5585545967
Test date:  2020-03-19 00:00:00
Postive percentage:  0.0
Negative percentage:  1.0

--------------------------------
Buy_list symbols:
[]
Short_list symbols:
['3988.HK', '1398.HK', '0939.HK', '0003.HK']
--------------------------------

  action   symbol
0  SHORT  3988.HK
1  SHORT  1398.HK
2  SHORT  0939.HK
3  SHORT  0003.HK


[SHORT] Symbol: 3988.HK Date open: 2020-03-20, Date limit: 2020-03-26, Traded price: 2.77.
Amount needed =  105260.0
Current cash =  875081.5585545967
Margin change =  52630.0
Cash balance =  980341.5585545967
Current margin:  142180.0

[SHORT] Symbol: 1398.HK Date open: 20

In [8]:
context.predictions

,date,num_predictions_long,num_predictions_short,percentage_long,percentage_short,predictions_long,predictions_short
0,2019-03-29,0.0,4.0,0.0,1.0,[],"[0386.HK, 0939.HK, 0941.HK, 1088.HK]"
1,2019-04-01,0.0,2.0,0.0,1.0,[],"[0941.HK, 1088.HK]"
2,2019-04-02,0.0,2.0,0.0,1.0,[],"[0941.HK, 1088.HK]"
3,2019-04-03,0.0,1.0,0.0,1.0,[],[3328.HK]
4,2019-04-04,0.0,0.0,0.0,0.0,[],[]
...,...,...,...,...,...,...,...
245,2020-03-26,0.0,0.0,0.0,0.0,[],[]
246,2020-03-27,0.0,0.0,0.0,0.0,[],[]
247,2020-03-30,0.0,0.0,0.0,0.0,[],[]
248,2020-03-31,0.0,0.0,0.0,0.0,[],[]


In [9]:
print('Time taken: {}'.format(str(dt.timedelta(seconds=(time.time() - start_time)))))

Time taken: 0:27:00.401601


In [10]:
results

,cash_value,date,porfolio_value,positions,positions_value
0,5.000000e+05,2019-03-29,500000.000000,[],0.0
1,7.487310e+05,2019-04-01,498520.988953,"[0386.HK, 1088.HK, 0939.HK]",-250210.0
2,1.066531e+06,2019-04-02,503410.988953,"[0386.HK, 1088.HK, 0939.HK, 0941.HK]",-563120.0
3,1.066531e+06,2019-04-03,494780.988953,"[0386.HK, 1088.HK, 0939.HK, 0941.HK]",-571750.0
4,1.395481e+06,2019-04-04,490720.988953,"[0386.HK, 1088.HK, 0939.HK, 0941.HK, 3328.HK]",-904760.0
...,...,...,...,...,...
245,6.681306e+05,2020-03-26,668130.552950,[],0.0
246,6.681306e+05,2020-03-27,668130.552950,[],0.0
247,6.681306e+05,2020-03-30,668130.552950,[],0.0
248,6.681306e+05,2020-03-31,668130.552950,[],0.0


In [11]:
#v10 - position size depending on impact on porfolio
#v11 - changed impact percentage to 0.001, re-adjusted number of positions - fixed biased to number of short trade
#v12 - change back to weight based position sizing. Keeping adjusted number of positions.
#v13 - SHORT ONLY positions. Result: better then SHORT and LONG - BUT it's very volatile (risky)
#v14 - RANDOM, as a benchmark
#v15 - SHORT only and give momentum_100 threshold to be < -30
#v16 - both Long and short - give momentum_100 threshold of 10 for long and -10 for short
#v17 - constraint on momemtum_50 instead
#v18 - no momentum threshold, and 5 positions max for long and short respectively
#v19 - a few thresholds for Negative, no long buy. thresholds:
#                                   and row['momentum_5']<-60 \
#                                   and row['close_scaled50']<1.5\
#                                   and row['volume_scaled50']>1\
#                                   and row['bull_ratio']<0:  (NO pREDICITONs AT ALL!!!!!!)
#v20 - Buy long, no threshold, limit to 10 in buying list max
#v21 - Short only. no threshold, limit to 10 in short list max NEED REDO THE CURVE
#v22 - Buy and Short, no threshold, limit to 5 in short and long respectively RESULT NOT BAD
#v23 - Buy and Short, no threshold, limit to 10 in short and long respectively
#v24 - fixed wrong algo of creating short list. Before, was all or nothing.
#    - added simulation of margin. for each trade, check if ELV > 0.5 of current margin.
#    - If not, drop the order.  Iteration order is all buys first and then all shorts. For each end of day,
#    - check again ELV > 0.5 of all stock value (with udpated price) - NO GAME OVER
#v25 - change amount shared between 5 stocks to 0.6 of capital_base (300,000 of 500,000), aiming to diversify more.
#v26 - use momentum_100 as criteria to sort predictions
#v27 - use momentum_50 as criteria to sort predictions
#v28 - use momentum_25 as criteria to sort predictions
#v29 - change amount shared between 5 stocks to 0.4 of capital_base (200,000 of 500,000), aiming to diversify more.
#    - change back to use volatility as criteria
#v31 - same as v29, but use momentum_100 as sort criteria
#v32 - same as v29, but use momentum_50 as sort criteria
#v33 - same as v29, but use momentum_5 as sort criteria
#v34 - same as v29, but use momentum_25 as sort criteria SO FAR BEST PERFORMANCE
#v35 - same as v29, but use volatility as sort criteria
#v36 - Revamped the position assigning alogo. Buy/Short predictions will be consolidated to form an alternating buy/short list.
#      Place order according to new parameters: max_daily_orders, max_positions, max_daily_margin_limit, etc
#      search criteria: momentum_25
#v37 - same as v36, but use volatility as sort criteria
#v38 - same as v36, but use momentum_100 as sort criteria
#v39 - same as v36, but use momentum_25 as sort criteria and FORCE EQUAL NUMBER OF BUY/SHORT
#v40 - same as v36, but use momentum_50 as sort criteria, removed FORCE EQUAL Number

version = 'v41'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH +VE MOMENTUM_50 FOR LONG, and -VE FOR SHORT

version = 'v42'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH +VE MOMENTUM_5 FOR LONG, and -VE FOR SHORT

version = 'v43'
#same as v36, but use momentum_5 as sort criteria, and ONLY take STOCK WITH +VE MOMENTUM_5 FOR LONG, and -VE FOR SHORT

version = 'v44'
#same as v36, but use momentum_5 as sort criteria, and ONLY take STOCK WITH +VE MOMENTUM_50 FOR LONG, and -VE FOR SHORT

version = 'v45'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH +VE BULL_RATIO FOR LONG, and -VE FOR SHORT

version = 'v46'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH >1 close_vs_50d_high FOR LONG, and <1 FOR SHORT

version = 'v47'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH >0 close_scaled50 FOR LONG, and <0 FOR SHORT

version = 'v48'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH >0 close_scaled5 FOR LONG, and <0 FOR SHORT
#with 5 additional close_scaled features

version = 'v49'
#same as v36, but use momentum_50 as sort criteria, and ONLY take STOCK WITH >0 close_scaled10 FOR LONG, and <0 FOR SHORT
#with 5 additional close_scaled features

version = 'v50'
#same as v36, but use momentum_50 as sort criteria, and ONLY take close_scaled5<0 FOR SHORT. No filter for LONG
#with 5 additional close_scaled features

version = 'v51'
#same as v36, but use momentum_50 as sort criteria No filter

version = 'v52'
#same as v36, but use momentum_50 as sort criteria No filter (SAME AS ABOVE. TEST RANDOMNESS) Results: NO RANDOMNESS

version = 'v53'
#same as v36, but it's Random Forrest, use momentum_50 as sort criteria No filter

version = 'v54'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled5>0. No filter for Short

version = 'v55'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled10>0. No filter for Short

version = 'v56'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled20>0. No filter for Short

version = 'v57'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled30>0. No filter for Short

version = 'v58'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled10>0. No filter for Short.
#RETRAIN ONLY EVERY 50 days
#

version = 'v59'
#same as v36, LGB V3, use momentum_50 as sort criteria, filter for LONG: close_scaled10>0. No filter for Short.
#RETRAIN ONLY EVERY 50 days
#

version = 'v60'
#same as v36, LGB V3, use momentum_50 as sort criteria No filter for both.
#RETRAIN ONLY EVERY 50 days
#

version = 'v61'
#same as v36, LGB V3, use momentum_50 as sort criteria No filter for both.

version = 'v62'
#same as v36, LGB V3, use momentum_25 as sort criteria, filter for LONG: close_scaled10>0. No filter for Short

version = 'v63'
#same as v36, LGB V3, use momentum_25 as sort criteria, filter for LONG: close_scaled10>0. close_scaled10<0 for short

version = 'v64'
#same as v36, LGB V3, use momentum_25 as sort criteria LONG only no filter, no short

version = 'v65'
#same as v36, LGB V3, use momentum_25 as sort criteria LONG only w close_scaled10>0 filter, no short

version = 'v66'
#same as v36, LGB V3, use momentum_25 as sort criteria SHORT only no filter

version = 'v67'
#same as v36, LGB V3, use momentum_25 as sort criteria SHORT only  w close_scaled10<0, NO LONG

version = 'v68'
#same as v36, LGB V3, use momentum_25 as sort criteria SHORT only  w close_scaled10<-0.5, NO LONG

#####################################################################################################################
#v30 - Same as v29 except if order cancelled due to less than minimum num lot, next stock on queue will move up and take its place.
#    - Buy_list will not be limited by a quota and will include all predictions. But number of orders will be limited to 5

save_csv(results, path_pc, 'LGB_v2_result_long_short_'+version+'_250.csv')

In [12]:
len(context.predictions.loc[context.predictions['num_predictions_long']>=context.predictions['num_predictions_short']]), len(context.predictions)

(37, 250)

In [13]:
outcomes_new.loc[('2019-05-15', '0386.HK')]['past_return_5_std50']

0.023174169304780747

In [14]:
save_csv(context.predictions, path_pc, 'predictions_v12_250.csv')

In [15]:
outcomes_new.loc[outcomes_new['bull_ratio']<0]

(close/ema)-1  (close/sma)-1  1-lower/close  50d_high  \
date       symbol                                                           
2000-01-04 0005.HK      -0.009161      -0.009346       0.009434  108.0000   
2000-01-05 0001.HK      -0.034979      -0.035667       0.036987   70.0424   
           0002.HK      -0.004171      -0.004255       0.004274   35.4000   
           0003.HK      -0.014278      -0.014565       0.014780    3.3297   
           0005.HK      -0.040417      -0.041335       0.019988  108.0000   
...                           ...            ...            ...       ...   
2020-04-17 2382.HK      -0.027032      -0.041699       0.168619  134.0000   
           2388.HK      -0.002752      -0.019886       0.173541   27.9500   
           2628.HK      -0.036590      -0.048205       0.203558   19.7800   
           3328.HK      -0.009745      -0.013387       0.086737    5.2200   
           3988.HK      -0.013242      -0.018316       0.063913    3.2300   

                    50d_high_volume     atr10  atr10/atr100  \
date       symbol                                             
2000-01-04 0005.HK        6895378.0  3.900000      0.977444   
2000-01-05 0001.HK        6058531.0  2.447860      1.125724   
           0002.HK        2993100.0  0.700000      1.147541   
           0003.HK       25881348.0  0.073350      1.132731   
           0005.HK       14657098.0  4.160000      1.036089   
...                             ...       ...           ...   
2020-04-17 2382.HK       27733878.0  5.994404      1.078749   
           2388.HK       42025767.0  0.751275      1.169316   
           2628.HK      290379202.0  0.557790      1.012916   
           3328.HK       73425627.0  0.124486      1.108433   
           3988.HK      766904691.0  0.066145      1.094492   

                    atr10/atr100_scaled50  atr10/atr20  atr10/atr20_scaled50  \
date       symbol                                                              
2000-01-04 0005.HK              -0.707107     0.987342             -0.707107   
2000-01-05 0001.HK               0.707107     1.066150              0.707107   
           0002.HK               0.707107     1.076923              0.707107   
           0003.HK               0.707107     1.069632              0.707107   
           0005.HK               1.067485     1.020233              1.067372   
...                                   ...          ...                   ...   
2020-04-17 2382.HK              -0.924712     0.927090             -1.314345   
           2388.HK               0.081783     0.998771             -0.891870   
           2628.HK              -0.555399     0.928371             -0.880266   
           3328.HK              -0.248713     0.954435             -1.028724   
           3988.HK              -0.939740     0.920030             -1.820771   

                    ...  volume_pct_chg_5_scaled50  volume_scaled50  \
date       symbol   ...                                               
2000-01-04 0005.HK  ...                        NaN         0.707107   
2000-01-05 0001.HK  ...                        NaN         0.707107   
           0002.HK  ...                        NaN         0.707107   
           0003.HK  ...                        NaN         0.707107   
           0005.HK  ...                        NaN         1.112075   
...                 ...                        ...              ...   
2020-04-17 2382.HK  ...                   1.131469        -0.334796   
           2388.HK  ...                   0.060324        -0.680581   
           2628.HK  ...                   0.698413         0.209339   
           3328.HK  ...                  -0.589103        -0.743785   
           3988.HK  ...                  -0.353722        -0.853307   

                    volume_slope  volume_std50  volume_vs_50d_high  \
date       symbol                                                    
2000-01-04 0005.HK  2.987214e+06           NaN            1.000000   
2000-01-05 0001.HK  2.8

In [16]:
len(outcomes_new)

206486